# Final Presentation Demo team kA

This shows step by step the abilities of our package


In [ ]:
import pandas as pd
import numpy as np
import pytest
from power_grid_model import ComponentType

from power_system_simulation.exceptions import *
from power_system_simulation.input_data_validation import *
from power_system_simulation.graph_processing import *
from power_system_simulation.power_grid_calculation import *

path_small_test_grid = "graph_cycle_disabled.json"

path_small_grid = "small_power_grid/input_network_data.json"
path_small_meta_data = "small_power_grid/meta_data.json"
path_small_p = "small_power_grid/active_power_profile.parquet"
path_small_q = "small_power_grid/reactive_power_profile.parquet"
path_small_ev = "small_power_grid/ev_active_power_profile.parquet"

path_large_grid = "large_power_grid/input_network_data.json"
path_large_meta_data = "large_power_grid/meta_data.json"
path_large_p = "large_power_grid/active_power_profile.parquet"
path_large_q = "large_power_grid/reactive_power_profile.parquet"
path_large_ev = "large_power_grid/ev_active_power_profile.parquet"


### Assignment 1: Graph processor

First, import a power_grid and create a graph out of it

In [ ]:
power_grid = load_grid_json(path_small_test_grid)
graph = create_graph(power_grid)

In [ ]:
"""
The small test grid layout:

1--<9>--2--[10]-3------(16)
^       |       .
20     [11]    [21]
        |       .
        4--[12]-5------(17)
        |       .
        [13]   [22]
        |       .
        6--[14]-7--[15]-8-----(19)
                |
               (18)

"""

### With this we can analyze some specifications and properties of the power-grid

Such as alternative lines when a specified line gets disabled.

In [ ]:
find_alternative_edges(graph, 11)

In [ ]:
find_alternative_edges(graph, 12)

In [ ]:
find_alternative_edges(graph, 15)

In [ ]:
try:
    find_alternative_edges(graph, 21)
except EdgeAlreadyDisabledError:
    print("This edge is already disabled")

And also any downstream vertices with respect to a source node.

In [ ]:
"""
The small test grid layout:

1--<9>--2--[10]-3------(16)
^       |       .
20     [11]    [21]
        |       .
        4--[12]-5------(17)
        |       .
        [13]   [22]
        |       .
        6--[14]-7--[15]-8-----(19)
                |
               (18)

"""

In [ ]:
find_downstream_vertices(graph, 10)

In [ ]:
find_downstream_vertices(graph, 11)

In [ ]:
graph.graph["source_node_id"] = 4

find_downstream_vertices(graph, 11)

### Assignment 2: Power Flow simulations

This is accomplished by using the PowerGrid class. This class contains the logic for running the model, and generates a graph internally for easy tracking and updating of the graph with respect to the power-grid data.

In [ ]:
power_grid = PowerGrid(path_small_grid, path_small_meta_data, path_small_p, path_small_q)
power_grid.run()

power_grid.line_summary

In [ ]:
power_grid.voltage_summary

In [ ]:
"""
The small grid layout:

                    3------(12)
                    |
                   [17]
                    |    
           /--[16]--2--[18]--4--[19]--5------(13)
          /                  .
0--<11>--1                  [24]
^         \                  .
10         \--[20]--6--[22]--8--[23]--9------(15)
                    |
                   [21]
                    |
                    7------(14)

"""

In [ ]:
find_alternative_edges(power_grid.graph, 22)

In [ ]:
find_alternative_edges(power_grid.graph, 11)

In [ ]:
"""
The small grid layout:

                    3------(12)
                    |
                   [17]
                    |    
           /--[16]--2--[18]--4--[19]--5------(13)
          /                  .
0--<11>--1                  [24]
^         \                  .
10         \--[20]--6--[22]--8--[23]--9------(15)
                    |
                   [21]
                    |
                    7------(14)

"""

In [ ]:
find_downstream_vertices(power_grid.graph, 11)

In [ ]:
find_downstream_vertices(power_grid.graph, 16)

In [ ]:
power_grid.graph.graph["source_node_id"] = 9

find_downstream_vertices(power_grid.graph, 20)

### Assignment 3: Features build on the PowerGrid class

Three features are designed to work on top with the data and functionality of the PowerGrid class

1. EV penetration level
    - Simulates a certain percentage of households with EV charges and calculates the additional load on the grid.
2. Optimal tap position
    - Calculates the optimal position of the transformer tap to optimize for one of the following 2 criteria:
        - Lowest average voltage deviation at the nodes
        - Lowest total energy loss in the lines
3. n-1 calculation
    - When a given line is disabled it will try to find an alternative line and calculate the power flow again. The results are returned in a table with a row for each alternative line, which stores the highest line load in the grid.

In [ ]:
power_grid = PowerGrid(path_large_grid, path_large_meta_data, path_large_p, path_large_q)
power_grid.run()

In [ ]:
power_grid.line_summary

In [ ]:
power_grid.voltage_summary

Firtst the ev penetration level

In [ ]:
ev_penetration = ev_penetration_level(power_grid, path_large_ev, 0.2)

In [ ]:
ev_penetration[0]

In [ ]:
ev_penetration[1]

In [ ]:
power_grid.voltage_summary.compare(ev_penetration[0])

In [ ]:
power_grid.line_summary.compare(ev_penetration[1])

Optimal tap position

In [ ]:
tap_position_1 = optimum_tap_position(power_grid, "minimal_deviation_u_pu")
print(tap_position_1)

In [ ]:
tap_position_2 = optimum_tap_position(power_grid, "minimal_energy_loss")
print(tap_position_2)

n-1 calculation

In [ ]:
n_1_calculation_1 = n_1_calculation(power_grid, 1353)
n_1_calculation_1

In [ ]:
n_1_calculation_2 = n_1_calculation(power_grid, 1354)
n_1_calculation_2